In [5]:
import os
import sys

from pathlib import Path

# Get the current working directory
cwd = Path(os.getcwd())

# Assuming your notebook is in the project root, set the project root as cwd
project_root = cwd.parent
print(project_root)

# If your notebook is in a subdirectory of the project root, you can modify the path accordingly:
# project_root = cwd.parent  # Go up one directory level
# project_root = cwd.parent.parent  # Go up two directory levels

# Add the project root directory to sys.path
sys.path.insert(0, str(project_root))
# sys.path.append('../')

import xpt.fit_analysis as xfa
import warnings
import corr_fitter.bs_utils as bs
import corr_fitter.bs_analysis as bs_analysis
import corr_fitter.load_data_priors as ld
import corr_fitter.corr_fit_analysis as fa
import xpt.priors as priors
import xpt.i_o as i_o
import xpt.fit_routine as fit
import xpt.plots as plots
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib
import tqdm
import h5py as h5
import yaml
import io
import lsqfit
import os
import importlib
import numpy as np
import gvar as gv
import pandas as pd
# from h5glance import H5Glance


warnings.simplefilter(action="default")
warnings.filterwarnings('ignore')


/home/gmoney/lqcd/hyperon-xpt-master


In [6]:
# Define paths and other variables
base_dir = "/home/gmoney/lqcd/hyperon-xpt-master"
input_dir = os.path.join(base_dir, "tests/input_files")
data_dir = os.path.join(base_dir, "data")
fit_results_dir = os.path.join(base_dir, "fit_results")
data_file = os.path.join(data_dir, "c51_2pt_octet_decuplet.h5")
hyperon_data_file = os.path.join(data_dir,"hyperon_data.h5")
bs_data_file = os.path.join(data_dir, "hyperon_bs_data.h5")
model_type = "all"
t_plot_min = 0
t_plot_max = 40
bs_N = 100
bs_seed = "seed"

In [3]:
importlib.reload(i_o)
input_output = i_o.InputOutput()
data = input_output.get_data()
print(data)
scale_setting_data = input_output.get_scale_setting_data()

['a06m310L', 'a09m135', 'a09m350', 'a09m400', 'a12m130', 'a12m220L', 'a12m220S', 'a12m310', 'a12m350', 'a12m400', 'a15m220', 'a15m310']
({'m_lambda': array([1255(11), 1260(10), 1303(10), 1090.9(8.0), 1123.2(8.2),
       1134.4(8.3), 1187.4(8.7), 1213.4(8.8), 1252.5(9.1), 1103.0(7.3),
       1162.1(7.7)], dtype=object), 'm_sigma': array([1311(12), 1312(11), 1346(11), 1186.1(8.7), 1208.5(8.8),
       1211.0(8.8), 1248.0(9.1), 1275.2(9.3), 1297.8(9.5), 1194.4(7.9),
       1236.8(8.2)], dtype=object), 'm_sigma_st': array([1501(13), 1571(13), 1600(13), 1461(11), 1480(11), 1481(11),
       1506(11), 1535(11), 1556(11), 1442.6(9.6), 1499(10)], dtype=object), 'm_xi_st': array([1642(15), 1625(13), 1671(13), 1535(11), 1555(11), 1558(11),
       1580(12), 1598(12), 1622(12), 1517(10), 1557(10)], dtype=object), 'm_xi': array([1410(13), 1388(11), 1411(11), 1271.4(9.3), 1282.4(9.4),
       1288.3(9.4), 1318.5(9.6), 1335.6(9.7), 1357.8(9.9), 1257.6(8.4),
       1294.3(8.6)], dtype=object), 'm_omega':

In [4]:
import pprint
import os.path
fit_results = {}

fit_results = {}
directory = 'fit_results/'

with h5.File('/home/gmoney/lqcd/data/c51_2pt_octet_decuplet.h5', "r") as f:
    ens = sorted(list(f.keys()))
    print(ens)
    for key in ens:
        file_path = directory + key + '/all/fit_params'
        if os.path.isfile(file_path):
            fit_results[key] = gv.load(file_path)
        else:
            print(f"File not found for key {key}")
pprint.pprint(fit_results)


['a06m220L', 'a06m310', 'a06m310L', 'a09m135', 'a09m220', 'a09m220_new_old', 'a09m220_o', 'a09m260', 'a09m310', 'a09m310_new_old', 'a09m310_o', 'a09m350', 'a09m400', 'a12m130', 'a12m180L', 'a12m180S', 'a12m220', 'a12m220L', 'a12m220S', 'a12m220XL', 'a12m220_ms', 'a12m220_new_old', 'a12m220_o', 'a12m260', 'a12m310', 'a12m310L', 'a12m310XL', 'a12m350', 'a12m400', 'a15m130', 'a15m135XL', 'a15m220', 'a15m260', 'a15m310', 'a15m310L', 'a15m310L_new_old', 'a15m310L_o', 'a15m350', 'a15m400']
File not found for key a06m220L
File not found for key a06m310
File not found for key a06m310L
File not found for key a09m135
File not found for key a09m220
File not found for key a09m220_new_old
File not found for key a09m220_o
File not found for key a09m260
File not found for key a09m310
File not found for key a09m310_new_old
File not found for key a09m310_o
File not found for key a09m350
File not found for key a09m400
File not found for key a12m130
File not found for key a12m180L
File not found for key 

In [ ]:
from tabulate import tabulate

abbreviations = [f[:-3] for f in os.listdir(input_dir) if f.endswith('.py')]

for abbr in abbreviations:
    fit_params = os.path.join(input_dir, f"{abbr}.py")
    if not os.path.exists(fit_params):
        print(f"Error: input file {fit_params} does not exist!")
        continue

    with open(fit_params, 'r') as f:
        input_file_contents = f.read()

    if 'p_dict' not in input_file_contents:
        print(f"Error: input file {fit_params} does not contain a dictionary called 'p_dict'!")
        continue

    try:
        p_dict = {}
        exec(input_file_contents, p_dict)
    except Exception as e:
        print(f"Error: Failed to execute the contents of input file {fit_params}!\n{str(e)}")
        continue

    if 'tag' not in p_dict:
        print(f"Warning: input file {fit_params} does not contain a dictionary called 'tag' within the 'p_dict' dictionary! Adding default values...")
        p_dict['tag'] = {
            'sigma' : 'sigma',
            'sigma_st' : 'sigma_st',
            'xi' :  'xi',
            'xi_st' : 'xi_st',
            'lam' : 'lam',
        }

    try:
        hyperon_fit = fa.analyze_hyperon_corrs(data_file, p_dict, model_type=model_type,
                                               bs=False, bs_file=bs_data_file,
                                               bs_path=abbr, bs_N=bs_N, bs_seed=bs_seed)
    except KeyError:
        print(f"KeyError: Error analyzing hyperons for input file {fit_params}. Skipping abbreviation {abbr}.")
        continue

    my_fit = hyperon_fit.get_fit()

    out_path = os.path.join(fit_results_dir, abbr, model_type)
    ld.pickle_out(fit_out=my_fit, out_path=out_path, species="hyperons")
    plot1 = hyperon_fit.return_best_fit_info()
    plot2 = hyperon_fit.plot_effective_mass(t_plot_min=t_plot_min, t_plot_max=t_plot_max, model_type=model_type,
                                            show_plot=True, show_fit=True)

    output_dir = os.path.join(fit_results_dir, abbr, f"{model_type}_{abbr}")
    os.makedirs(output_dir, exist_ok=True)
    output_pdf = os.path.join(output_dir, 'output.pdf')
    with PdfPages(output_pdf) as pp:
        pp.savefig(plot1)
        pp.savefig(plot2)

    params_df = pd.DataFrame(my_fit.p).transpose()

    print("Abbreviation:", abbr)
    print(tabulate(params_df, headers='keys', tablefmt='fancy_grid'))

    # Ask the user if the fit result is acceptable
    while True:
        result = input(f"Is the fit result for abbreviation {abbr} acceptable? (y/n): ")
        if result == "y":
            # Save the result and skip this fit in future runs
            out_path = 'fit_results/{0}/{1}/'.format(abbr, model_type)
            ld.pickle_out(fit_out=my_fit, out_path=out_path, species="hyperons")
            break
        elif result == "n":
            break
